In [ ]:
import torch

import torch.nn as nn

In [ ]:
# Corpus and Vocabulary

corpus = "deep learning is fun and deep learning is powerful"

words = corpus.split()

vocab = sorted(set(words))

word_to_idx = {word: i for i , word in enumerate(vocab)}

idx_to_word = {i: word for word, i in word_to_idx.items()}

vocab_size = len(vocab)

In [ ]:
# Training Sequences
training_sentences = [
    ['deep', 'learning', 'is', 'fun'],
    ['learning', 'is', 'powerful'],
    ['deep', 'learning', 'is', 'powerful'],
    ['learning', 'is', 'fun', 'and', 'powerful']
]

In [ ]:
seq_length = 3

input_seqs = []

target_words = []

In [ ]:
for sentence in training_sentences:

    if len(sentence) <= seq_length:

        continue

    for i in range(len(sentence) - seq_length):

        input_seq = sentence[i: i + seq_length]

        target_word = sentence[i + seq_length]

        input_tensor = torch.tensor([word_to_idx[word]for word in input_seq], dtype=torch.long)

        target_tensor = torch.tensor(word_to_idx[target_word], dtype=torch.long)

        input_seqs.append(input_tensor)

        target_words.append(target_tensor)

In [ ]:
# LSTM Model

class WordLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(WordLSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim,hidden_dim, batch_first=True)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    
    def forward(self, x, hidden):

        embedded = self.embedding(x)

        out, hidden = self.lstm(embedded, hidden)

        out = self.fc(out[:,-1,:])

        return out, hidden

In [ ]:
# Hyperparameters

embedding_dim = 10

hidden_dim = 16

num_epochs = 300

learning_rate = 0.01

In [ ]:
model = WordLSTM(vocab_size, embedding_dim, hidden_dim)

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)